<a href="https://colab.research.google.com/github/ik4Rus/blog/blob/master/tip_07_building_asynch_client_server_applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Python Tip 7: Building Asynchronous Client-Server Applications with Python's Selectors Module

One of the key challenges in building network applications is handling I/O operations in a scalable and efficient manner. Python's selectors module provides a powerful solution to this challenge, allowing developers to build asynchronous client-server applications that can handle multiple clients simultaneously. In this blog post, we will explore the use of the selectors module in Python and demonstrate how to build an asynchronous client-server application with it.

*   **Twitter**: 🐍💻 Looking to build scalable and efficient network applications with Python? Check out my latest blog post on using the selectors module to create asynchronous client-server apps! 🤖🚀 #PythonProgramming #AsyncProgramming <bloglink>
*   **LinkedIn**:
Are you a Python developer looking to build scalable and efficient network applications? 🐍💻 In my latest blog post, I explore how to use the selectors module in Python to build asynchronous client-server applications that can handle multiple clients simultaneously! 🤖🚀
The selectors module provides a powerful solution to the challenge of handling I/O operations in a scalable and efficient manner. By using the techniques demonstrated in this blog post, you can build your own asynchronous client-server applications with Python and take your network programming skills to the next level! Check out the post below 👇 and let me know what you think! #PythonProgramming #AsyncProgramming <bloglink>


## Introduction: Using the Selectors Module for Asynchronous Client-Server Applications

Asynchronous programming has become increasingly important for building high-performance network applications. In the context of client-server applications, it allows us to handle multiple client connections without blocking the main program flow, making our application more efficient and scalable. In this blog post, we'll explore how to use the selectors module in Python to create an asynchronous client-server application. The selectors module provides a powerful and efficient way to monitor multiple sockets and file objects for events, such as incoming data, and handle them in an asynchronous manner. Let's dive in and learn how to use the selectors module for building advanced network applications in Python.

## Problem Statement: Handling Multiple Client Connections in a Synchronous Way

When building a client-server application, we often face the challenge of handling multiple client connections simultaneously. In a synchronous approach, the server waits for a client to connect, processes its request, and then moves on to the next client. This approach, however, can lead to blocking and slow down the server's response time, especially when dealing with a large number of clients.

Consider the following example of a synchronous server that handles client connections one at a time:

In [ ]:
import socket

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 65432        # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    while True:
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            data = conn.recv(1024)
            conn.sendall(data)

In the synchronous server example, the server uses the s.listen() method to listen for incoming connections from clients. Once a client connects to the server using the s.accept() method, the server creates a new socket object conn to communicate with the client. The conn.recv(1024) method is used to receive data from the client. The 1024 parameter specifies the maximum amount of data that can be received at once, which is commonly used as a buffer size. The server then sends a response back to the client using the conn.sendall(data) method. Finally, the server waits for the next client to connect by calling s.accept() again.

Note that in a synchronous server, the server can only handle one client at a time. While the server is waiting for a client to send data using conn.recv(1024), it is blocked and unable to handle any other client connections. This can cause the server to become unresponsive if it is handling many clients simultaneously. In the next section, we'll explore how to use the selectors module in Python to handle multiple client connections asynchronously, improving our server's performance and scalability.


If you want to test the code, you can execute it in a first python terminal while starting the following code in the second. 

In [ ]:
import socket

HOST = '127.0.0.1'  # The server's hostname or IP address
PORT = 65432        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print(f"Connected to server on port {s.getsockname()[1]}")
    s.sendall(b'Hello, world')
    data = s.recv(1024)

print('Received', repr(data))

You should now see in both processes that the same port is connected to our socket, and that our 'Hello, world' data is returned.

While this approach works for a small number of clients, it can quickly become a bottleneck when dealing with many clients. In the next section, we'll explore how to use the selectors module in Python to handle multiple client connections asynchronously, improving our server's performance and scalability.

## Using the selectors module for an asynchronous client-server application

Python's selectors module provides a way to efficiently handle multiple socket connections in an asynchronous manner, allowing a server to handle multiple clients simultaneously without being blocked by individual client requests. Here's how to use the selectors module to create an asynchronous client-server application:

1.) First, import the necessary modules and create a Selector object:


In [ ]:
import selectors
import socket

sel = selectors.DefaultSelector()

2.) Define a function to accept incoming connections from clients:

In [ ]:
def accept_wrapper(sock):
    conn, addr = sock.accept()
    print(f"Accepted connection from {addr}")
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, data=None)

This function creates a new socket object `conn` and registers it with the selector for reading events. It also sets the socket to non-blocking mode so that it does not block while waiting for data.

3.) Define a function to handle incoming client requests:

In [ ]:
def service_connection(key, mask):
    sock = key.fileobj
    data = key.data
    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)
        if recv_data:
            print(f"Received {repr(recv_data)} from {sock.getpeername()}")
            data.outb += recv_data
        else:
            print(f"Closing connection to {sock.getpeername()}")
            sel.unregister(sock)
            sock.close()
    if mask & selectors.EVENT_WRITE:
        if data.outb:
            print(f"Echoing {repr(data.outb)} to {sock.getpeername()}")
            sent = sock.send(data.outb)
            data.outb = data.outb[sent:]

This function handles incoming client requests by receiving data from the client using `sock.recv(1024)`, and sending a response back to the client using `sock.send()`. It also manages the data object associated with the socket, which stores any outgoing data to be sent to the client.

4.) Define a function to start the server and listen for incoming connections:

In [ ]:
def start_server():
    HOST = '127.0.0.1'
    PORT = 65432
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind((HOST, PORT))
    sock.listen()
    print(f"Listening on {HOST}:{PORT}")
    sock.setblocking(False)
    sel.register(sock, selectors.EVENT_READ, data=None)

This function creates a new socket object, binds it to a specific host and port, and listens for incoming connections using `sock.listen()`. It also registers the socket with the selector for reading events.

5.) Finally, create a loop to continuously monitor the selector for incoming events and handle them asynchronously:

In [ ]:
def event_loop():
    while True:
        events = sel.select(timeout=None)
        for key, mask in events:
            if key.data is None:
                accept_wrapper(key.fileobj)
            else:
                service_connection(key, mask)

This loop continuously monitors the selector for incoming events using sel.`select()`, and calls the appropriate handler function based on the type of event received. If the event is a new connection, it calls `accept_wrapper()` to accept the connection and register it with the selector. If the event is a read or write event on an existing connection, it calls `service_connection()` to handle the incoming data and send any outgoing data.

## Summary

In summary, by using the selectors module in Python, we can create an asynchronous client-server application that allows for multiple clients to connect to a server and communicate with it simultaneously. This allows for more efficient and scalable network communication in Python applications. The selectors module provides a powerful and flexible way to handle I/O multiplexing in Python, and by using it, we can avoid the limitations of traditional blocking I/O operations. With the code examples provided, you should be able to start building your own asynchronous client-server applications in Python.